In [1]:
import itertools
from collections import Counter, namedtuple

In [2]:
def get_types():
    return ',Single,Pair,Three of a kind,Four of a kind,Five of a kind,Maxi Yahtzee'.split(',')

In [3]:
def get_dice_frequencies(throw, types):
    """
    Returns the counts of unique dice, pairs, three of a kind etc
    """
    c = Counter(throw)
    frequencies = {}
    for i in [str(j) for j in range(1, 7)]:
        if i in c:
            result_type = types[c[i]]
            if result_type in frequencies:
                frequencies[result_type].append(i)
            else:
                frequencies[result_type] = [i]
    return frequencies

In [4]:
def dicesum(throw):
    """
    Returns the sum of all dices in a throw
    """
    return sum([int(i) for i in throw])

In [5]:
def get_throw_result(throw, types):
    """
    Determines the type of throw:
    Full/Big/Small straight
    Maxi Yahtzee, 5/4/3 equal
    3/2/1 pair
    Tower and Castle
    To Do: number of 1's, 2's, 3's, ... for the top part of the scoring table
    """
    
    # check input
    assert type(throw) is str, 'String expected, but "%r" is %r' % (throw, type(throw))
    assert len(throw) == 6, 'Throw expected to be 6 characters, not %i' % len(throw)
    assert set('123456') == set('123456' + throw), 'Only dices between 1 and 6 allowed, found "%s"' % throw

    # process
    throw_type = ''
    throw_results = []
    if throw == '123456':
        throw_results.append(result('Full straight', '123456', 21))
    if ''.join(sorted(set(throw)))[0:5] == '12345':
        throw_results.append(result('Small straight', '12345', 15))
    if ''.join(sorted(set(throw)))[-5:] == '23456':
        throw_results.append(result('Big straight', '23456', 20))

    # Collect dice frequencies
    dice_frequencies = get_dice_frequencies(throw, types)
    # test for pairs, three of a kind, ..., Maxi Yahtzee
    for f in range(2,7):
        result_type = types[f]
        if result_type in dice_frequencies:
            for dice in dice_frequencies[result_type]:
                throw_results.append(result(result_type, dice, f * int(dice)))
    # test for Tower, Castle, Two pair, Three pair
    if 'Pair' in dice_frequencies and len(dice_frequencies['Pair']) == 3:
        throw_results.append(result('Three pair', throw, dicesum(throw)))
    if 'Three of a kind' in dice_frequencies and len(dice_frequencies['Three of a kind']) == 2:
        throw_results.append(result('Castle (3 + 3)', throw, dicesum(throw)))
    if 'Four of a kind' in dice_frequencies and 'Pair' in dice_frequencies:
        throw_results.append(result('Tower (4 + 2)', throw, dicesum(throw)))
    if len(throw_results) < 1 :
        raise Exception("Unknown combination of dice: " , throw)
    return throw_results

In [6]:
def test_dice_frequencies_singles():
    assert get_dice_frequencies('123456', types = get_types()) == {'Single': ['1', '2', '3', '4', '5', '6']}
def test_dice_frequencies_3():
    assert get_dice_frequencies('222333', types = get_types()) == {'Three of a kind': ['2', '3']}
def test_dice_frequencies_4():
    assert get_dice_frequencies('111122', types = get_types()) == {'Four of a kind': ['1'], 'Pair': ['2']}
def test_dice_frequencies_6():
    assert get_dice_frequencies('111111', types = get_types()) == {'Maxi Yahtzee': ['1']}

In [7]:
def test_Tower():
    assert get_throw_result('111122', types = get_types()) == [result(type='Pair', dice='2', score=4),
                                          result(type='Four of a kind', dice='1', score=4),
                                          result(type='Tower (4 + 2)', dice='111122', score=8)]
def test_Three_pairs():
    assert get_throw_result('112244', types = get_types()) == [result(type='Pair', dice='1', score=2),
                                          result(type='Pair', dice='2', score=4),
                                          result(type='Pair', dice='4', score=8),
                                          result(type='Three pair', dice='112244', score=14)]
def test_Five_of_a_kind():
    assert get_throw_result('444441', types = get_types()) == [result(type='Five of a kind', dice='4', score=20)]
def test_Maxi_yahtzee():
    assert get_throw_result('222222', types = get_types()) == [result(type='Maxi Yahtzee', dice='2', score=12)]

In [8]:
def test_dice_too_low_high_fail():
    try:
        t = get_throw_result('012345', types = get_types())
    except AssertionError, e:
        assert e.message == 'Only dices between 1 and 6 allowed, found "012345"', e.message
def test_dice_as_int_not_string_fail():
    try:
        t = get_throw_result(123456, types = get_types())
    except AssertionError, e:
        assert e.message == 'String expected, but "123456" is <type \'int\'>', e.message
def test_dice_too_short_fail():
    try:
        t = get_throw_result('12345', types = get_types())
    except AssertionError, e:
        assert e.message == 'Throw expected to be 6 characters, not 5', e.message

In [10]:
result = namedtuple('result', ['type', 'dice', 'score'])
if __name__ == '__main__':
    types = get_types() #',Single,Pair,Three of a kind,Four of a kind,Five of a kind,Maxi Yahtzee'.split(',')
    dices = [i for i in range(1, 7)]
    throws = {}
    # the code below is overkill in that it generates all 46656 combinations of 6 dice throws
    # but the resulting unique set is 462 combination, which seems to be correct
    for i in itertools.product(dices, dices, dices, dices, dices, dices):
        throw = ''.join([str(j) for j in sorted(i)])
        throws[throw] = ''
    # assing score to each throw
    for throw in sorted(throws):
        throw_results = get_throw_result(throw, types)
        print throw, [(t.type, t.dice, t.score) for t in throw_results]

111111 [('Maxi Yahtzee', '1', 6)]
111112 [('Five of a kind', '1', 5)]
111113 [('Five of a kind', '1', 5)]
111114 [('Five of a kind', '1', 5)]
111115 [('Five of a kind', '1', 5)]
111116 [('Five of a kind', '1', 5)]
111122 [('Pair', '2', 4), ('Four of a kind', '1', 4), ('Tower (4 + 2)', '111122', 8)]
111123 [('Four of a kind', '1', 4)]
111124 [('Four of a kind', '1', 4)]
111125 [('Four of a kind', '1', 4)]
111126 [('Four of a kind', '1', 4)]
111133 [('Pair', '3', 6), ('Four of a kind', '1', 4), ('Tower (4 + 2)', '111133', 10)]
111134 [('Four of a kind', '1', 4)]
111135 [('Four of a kind', '1', 4)]
111136 [('Four of a kind', '1', 4)]
111144 [('Pair', '4', 8), ('Four of a kind', '1', 4), ('Tower (4 + 2)', '111144', 12)]
111145 [('Four of a kind', '1', 4)]
111146 [('Four of a kind', '1', 4)]
111155 [('Pair', '5', 10), ('Four of a kind', '1', 4), ('Tower (4 + 2)', '111155', 14)]
111156 [('Four of a kind', '1', 4)]
111166 [('Pair', '6', 12), ('Four of a kind', '1', 4), ('Tower (4 + 2)', '1111